In [3]:
def quantize_vertical_pytorch(tensor, quant_group_size):
    """
    Quantizes a tensor into a vertically tiled format compatible with dequantize_vertical_pytorch.
    tensor: (k, n) float tensor to quantize
    m: unused (kept for signature compatibility)
    n: number of columns
    quant_group_size: size of quantization groups along k dimension
    Returns:
        tensor_quant: (k//2, n) quantized tensor (uint8, two 4-bit values per byte)
        scale: (k//quant_group_size, n) scaling factors
    """
    k,n = tensor.shape
    assert k % 2 == 0, "k must be even for vertical quantization"
    assert k % quant_group_size == 0, "k must be divisible by quant_group_size"

    # Compute scales per group
    num_groups = k // quant_group_size
    tensor_groups = tensor.reshape(num_groups, quant_group_size, n)
    scale = torch.max(torch.abs(tensor_groups), dim=1)[0] / 7.0  # Max value for int4 is 7
    scale = scale.clamp(min=1e-6)  # Avoid division by zero

    # Expand scales to match tensor shape
    scale_expanded = torch.repeat_interleave(scale, quant_group_size, dim=0)

    # Quantize to int8 range (-8 to 7)
    tensor_scaled = tensor / scale_expanded
    tensor_int8 = torch.round(tensor_scaled.clamp(-8, 7)).to(torch.int8)

    # Convert to uint4 range (0-15: -8 to -1 becomes 8-15, 0-7 stays 0-7)
    tensor_uint4 = torch.where(tensor_int8 < 0, tensor_int8 + 16, tensor_int8).to(torch.int8)

    # Pack two 4-bit values into one byte
    tensor_uint4 = tensor_uint4.reshape(k // 2, 2, n)  # Group pairs along k
    low_bits = tensor_uint4[:, 0, :] & 0x0F
    high_bits = (tensor_uint4[:, 1, :] & 0x0F) << 4
    tensor_quant = (high_bits | low_bits).to(torch.int8)

    return tensor_quant, scale , tensor.shape

def quantize_horizontal_pytorch(tensor, quant_group_size):
    """
    Quantizes a tensor into a horizontally tiled format compatible with dequantize_horizontal_pytorch.
    tensor: (m, k) float tensor to quantize
    m: number of rows
    k: number of columns
    quant_group_size: size of quantization groups along k dimension
    Returns:
        tensor_quant: (m, k//2) quantized tensor (uint8, two 4-bit values per byte)
        scale: (m, k//quant_group_size) scaling factors
    """
    m,k = tensor.shape
    assert k % 2 == 0, "k must be even for horizontal quantization"
    assert k % quant_group_size == 0, "k must be divisible by quant_group_size"

    # Compute scales per group
    num_groups = k // quant_group_size
    tensor_groups = tensor.reshape(m, num_groups, quant_group_size)
    scale = torch.max(torch.abs(tensor_groups), dim=2)[0] / 7.0  # Max value for int4 is 7
    scale = scale.clamp(min=1e-6)  # Avoid division by zero

    # Expand scales to match tensor shape
    scale_expanded = torch.repeat_interleave(scale, quant_group_size, dim=1)

    # Quantize to int8 range (-8 to 7)
    tensor_scaled = tensor / scale_expanded
    tensor_int8 = torch.round(tensor_scaled.clamp(-8, 7)).to(torch.int8)

    # Convert to uint4 range (0-15: -8 to -1 becomes 8-15, 0-7 stays 0-7)
    tensor_uint4 = torch.where(tensor_int8 < 0, tensor_int8 + 16, tensor_int8).to(torch.int8)

    # Pack two 4-bit values into one byte
    tensor_uint4 = tensor_uint4.reshape(m, k // 2, 2)  # Group pairs along k
    low_bits = tensor_uint4[:, :, 0] & 0x0F
    high_bits = (tensor_uint4[:, :, 1] & 0x0F) << 4
    tensor_quant = (high_bits | low_bits).to(torch.int8)

    return tensor_quant, scale , tensor.shape


def dequantize_vertical_pytorch(tensor_quant, m, n, scale, quant_group_size):
    """
    Dequantizes a vertically tiled quantized tensor similar to dequant_vertical.
    tensor_quant: (k//2, n) quantized tensor (uint8, two 4-bit values per byte)
    scale: (k//quant_group_size, n) scaling factors
    """
    k = 2 * tensor_quant.shape[0]  # Full dimension after unpacking
    # Convert to int16 to prevent overflow during bit operations
    tensor_quant_int16 = tensor_quant.to(torch.int16)
    
    # Unpack all 4-bit values at once
    # Reshape to (k//2, n, 1) and expand to separate high and low bits
    low_bits = tensor_quant_int16 & 0x0F  # Lower 4 bits
    high_bits = (tensor_quant_int16 >> 4) & 0x0F  # Upper 4 bits
    
    # Stack and reshape: interleave low and high bits
    tensor = torch.stack([low_bits, high_bits], dim=1).reshape(k, n)
    
    # Convert to signed int8 (0-7 stay as is, 8-15 become -8 to -1)
    tensor = torch.where(tensor < 8, tensor, tensor - 16).to(torch.int8)
    
    # Apply scales
    scale_expanded = torch.repeat_interleave(scale, quant_group_size, dim=0)[:k]
    tensor = tensor.to(torch.float16) * scale_expanded
    
    return tensor

def dequantize_horizontal_pytorch(tensor_quant, m, k, scale, quant_group_size):
    """
    Dequantizes a horizontally tiled quantized tensor similar to dequant_horizontal.
    tensor_quant: (m, k//2) quantized tensor (uint8, two 4-bit values per byte)
    scale: (m, k//quant_group_size) scaling factors
    """
    # Convert to int16 to prevent overflow during bit operations
    tensor_quant_int16 = tensor_quant.to(torch.int16)
    
    # Unpack all 4-bit values at once
    low_bits = tensor_quant_int16 & 0x0F  # Lower 4 bits
    high_bits = (tensor_quant_int16 >> 4) & 0x0F  # Upper 4 bits
    
    # Stack and reshape: interleave low and high bits
    tensor = torch.stack([low_bits, high_bits], dim=2).reshape(m, k)
    
    # Convert to signed int8
    tensor = torch.where(tensor < 8, tensor, tensor - 16).to(torch.int8)
    
    # Apply scales
    scale_expanded = torch.repeat_interleave(scale, quant_group_size, dim=1)[:, :k]
    tensor = tensor.to(torch.float16) * scale_expanded
    
    return tensor

In [9]:

@triton.jit
def dequant_vertical(
    tensor_ptr,
    bn,bk,
    stride,
    tensor_startpoint,
    m,n , 
    scale_ptr,
    scale_start,
    scale_group_size,
):
    offset_tensor = tensor_startpoint + tl.expand_dims(tl.arange(0,bn) , 0)+(stride)*(tl.expand_dims(tl.arange(0,bk) , 1)//2)
    offset_tensor_mask=tensor_startpoint+(n)*((tl.expand_dims(tl.arange(0,bk) , 1))//2+1)
    offset_tensor_mask=tl.where(offset_tensor_mask<m*n//2 ,offset_tensor_mask , m*n//2 )
    

    # print(offset_tensor , offset_tensor<offset_tensor_mask)
    offset_scale=scale_start + tl.expand_dims(tl.arange(0,bn) , 0)
    offset_scale_mask=scale_start + min(bn,n)
    
    tensor=tl.load(tensor_ptr+offset_tensor  ,offset_tensor<offset_tensor_mask , 0)
    scale=tl.load(scale_ptr+offset_scale  ,offset_scale<offset_scale_mask , 0)
    
    shifter=(0*tl.expand_dims(tl.arange(0,bn) , 0)+(tl.expand_dims(tl.arange(0,bk) , 1))%2)*4
    tensor = (tensor >> shifter) & 0x0F 
    tensor=tl.where(tensor<8 , tensor , tensor-16).to(tl.int8)
    # print("vertical" , tensor , scale)
    return tensor,scale


@triton.jit
def dequant_horizontal(
    tensor_ptr,
    bm , bk,
    stride,
    tensor_startpoint,
    m,n,
    scale_ptr,
    scale_start,
    scale_group_size
):
    offset_tensor=tensor_startpoint+(stride//2)*(tl.expand_dims(tl.arange(0,bm) , 1)) + tl.expand_dims(tl.arange(0,bk), 0)//2
    offset_tensor_mask=tensor_startpoint+stride//2*(tl.expand_dims(tl.arange(1,bm+1) , 1))
    offset_tensor_mask=tl.where(offset_tensor_mask<m*n//2 ,offset_tensor_mask , m*n//2 )
    
    offset_scale=scale_start+(tl.cdiv(stride,scale_group_size)*(tl.expand_dims(tl.arange(0,bm) , 1)))
    offset_scale_mask=scale_start+(tl.cdiv(stride,scale_group_size)*(tl.expand_dims(tl.arange(1,bm+1) , 1)))
    offset_scale_mask=tl.where(offset_scale_mask<m*tl.cdiv(stride,scale_group_size) ,offset_scale_mask , m*(tl.cdiv(stride,scale_group_size) ))
    
    # print(offset_scale , offset_scale_mask)
    tensor=tl.load(tensor_ptr+offset_tensor  ,offset_tensor<offset_tensor_mask , 0)
    scale=tl.load(scale_ptr+offset_scale  ,offset_scale<offset_scale_mask, 0)
    shift_offset=bk*(tl.expand_dims(tl.arange(0,bm) , 1)) + tl.expand_dims(tl.arange(0,bk), 0)
    shifter = (shift_offset % 2) * 4
    
    tensor = (tensor >> shifter) & 0x0F 
    tensor=tl.where(tensor<8 , tensor , tensor-16).to(tl.int8)
    # print("horizontal" ,  tensor , scale)
    return tensor , scale



@triton.jit
def kernel2(
    xl1_ptr , x1_quant_ptr , x1_scale_ptr , w_quant_ptr , w_scale_ptr , l2_ptr , output_ptr,
    m,n,k ,r,
    bm: tl.constexpr, bn: tl.constexpr, bk: tl.constexpr, group_sz: tl.constexpr , quant_group_size: tl.constexpr
):
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)
    num_pid_m, num_pid_n = tl.num_programs(0), tl.num_programs(1)
    pid_m_new , pid_n_new=tl.swizzle2d(pid_m, pid_n, num_pid_m, num_pid_n, group_sz)  # Weirdness: tl.swizzle2d doesn't work when simulating on CPU
    
    acc = tl.zeros((bm, bn), dtype=tl.float32)
    
    
    # x_ptr_offset=k*(pid_m_new*bm + tl.expand_dims(tl.arange(0,bm), 1))+ tl.expand_dims(tl.arange(0,bk), 0)
    # x_ptr_mask=k*tl.expand_dims(pid_m_new*bm+tl.arange(1,bm+1),1)
    # x_ptr_mask=tl.where(x_ptr_mask<m*k , x_ptr_mask , m*k)
    # x_ptr_scale_offset=k*(pid_m_new*bm + tl.expand_dims(tl.arange(0,bm), 1))+ tl.expand_dims(tl.arange(0,bk), 0)
    
    # w_ptr_offset=(pid_n_new*bn+tl.expand_dims(tl.arange(0,bn), 0))+tl.expand_dims(tl.arange(0,bk), 1)*n
    # w_mask=(k-1)*n+pid_n_new*bn+tl.expand_dims(tl.arange(1,bn+1), 0)
    # w_mask=tl.where(l1_mask<k*n , l1_mask , k*n)

    xl1_ptr_offset=r*(pid_m_new*bm + tl.expand_dims(tl.arange(0,bm), 1))+ tl.expand_dims(tl.arange(0,bk), 0)
    x_l1_mask=r*tl.expand_dims(pid_m_new*bm+tl.arange(1,bm+1),1)
    x_l1_mask=tl.where(x_l1_mask<m*r , x_l1_mask , m*r)
    
    l2_offset=(pid_n_new*bn+tl.expand_dims(tl.arange(0,bn), 0))+tl.expand_dims(tl.arange(0,bk), 1)*n
    l2_mask=pid_n_new*bn+tl.expand_dims(tl.arange(1,bk+1), 1)*n
    l2_mask=tl.where(l2_mask<r*n , l2_mask , r*n)

    # print(l2_offset)
    startpoint_x=k//2*(pid_m_new*bm)
    startpoint_x_scale=pid_m_new*bm*tl.cdiv(k,quant_group_size)
    
    startpoint_w=pid_n_new*bn
    startpoint_w_scale=pid_n_new*bn
    
    acc = tl.zeros((bm, bn), dtype=tl.float32)
    zeros = tl.zeros((bm, bn), dtype=tl.float32)
    # print('pid_m_new' , pid_m_new,'pid_n_new',pid_n_new)
    # print("w_start" , startpoint_w )
    # print("startpoint_w_scale" , startpoint_w_scale )
    # print("x_start" , startpoint_x)
    for i in range ( 0 , tl.cdiv(k , bk)):
        
        dequant_x1 , scale_x1=dequant_horizontal(x1_quant_ptr ,  bm , bk , k ,startpoint_x , m , k ,x1_scale_ptr , startpoint_x_scale+(i*bk)//quant_group_size ,  quant_group_size)
        
        dequant_w , scale_w=dequant_vertical(w_quant_ptr ,  bk , bn , n ,startpoint_w , k , n , w_scale_ptr ,startpoint_w_scale+n*((i*bk)//quant_group_size) , quant_group_size)
    
        acc_x_w=  tl.dot(dequant_x1, dequant_w)
        # print(acc_x_w)
        acc_x_scale_w_scale = scale_x1*scale_w
        startpoint_x+=bk//2
        startpoint_w+=bk*n//2
        if i*bk<r:
            # print(xl1_ptr_offset , xl1_ptr_offset<x_l1_mask)
            xl1_loaded = tl.load(xl1_ptr+xl1_ptr_offset , xl1_ptr_offset<x_l1_mask , 0)
            l2_loaded = tl.load(l2_ptr+l2_offset , l2_offset<l2_mask , 0)
            # print(xl1_loaded , l2_loaded)
            acc_lora=  tl.dot(xl1_loaded, l2_loaded)
            acc+=acc_lora
            xl1_ptr_offset+=bk
            l2_offset+=n*bk
            l2_mask+=n*bk
            l2_mask=tl.where(l2_mask<r*n , l2_mask , r*n)
        acc+=tl.fma(acc_x_w , acc_x_scale_w_scale , zeros)
    
    answer_offset=n*(pid_m_new*bm + tl.expand_dims(tl.arange(0,bm), 1))+ pid_n_new*bn+tl.expand_dims(tl.arange(0,bn), 0)
    answer_mask=n*(pid_m_new*bm + tl.expand_dims(tl.arange(0,bm), 1)+1)
    answer_mask=tl.where(answer_mask<m*n , answer_mask , m*n)
    acc=acc.to(tl.float16)
    tl.store(output_ptr+answer_offset , acc , answer_offset<answer_mask)    



In [8]:
import torch

def cdiv(a,b):
    return (a+b-1) //b
    
import torch.nn as nn
import triton
import triton.language as tl
import gc


@triton.jit
def kernel1(
    x_ptr, l1_ptr, xl1_ptr, x_quant_ptr, scale_ptr,
    m, n, k,
    bm: tl.constexpr, bn: tl.constexpr, bk: tl.constexpr, group_sz: tl.constexpr, bk_quant: tl.constexpr, group_size: tl.constexpr
):
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)
    num_pid_m, num_pid_n = tl.num_programs(0), tl.num_programs(1)
    pid_m_new, pid_n_new = tl.swizzle2d(pid_m, pid_n, num_pid_m, num_pid_n, group_sz)

    # Matmul: X @ l1
    acc = tl.zeros((bm, bn), dtype=tl.float32)
    x_ptr_offset = k * (pid_m_new * bm + tl.expand_dims(tl.arange(0, bm), 1)) + tl.expand_dims(tl.arange(0, bk), 0)
    l1_ptr_offset = (pid_n_new * bn + tl.expand_dims(tl.arange(0, bn), 0)) + tl.expand_dims(tl.arange(0, bk), 1) * n
    x_ptr_mask = k * tl.expand_dims(pid_m_new * bm + tl.arange(1, bm + 1), 1)
    x_ptr_mask = tl.where(x_ptr_mask < m * k, x_ptr_mask, m * k)
    l1_mask = (k - 1) * n + pid_n_new * bn + tl.expand_dims(tl.arange(1, bn + 1), 0)
    l1_mask = tl.where(l1_mask < k * n, l1_mask, k * n)
    for i in range(0, tl.cdiv(k, bk)):
        x_loaded = tl.load(x_ptr + x_ptr_offset, x_ptr_offset < x_ptr_mask, 0)
        l1_loaded = tl.load(l1_ptr + l1_ptr_offset, l1_ptr_offset < l1_mask, 0)
        acc += tl.dot(x_loaded, l1_loaded)
        x_ptr_offset += bk
        l1_ptr_offset += n * bk
    xl1_offset = n * (pid_m_new * bm + tl.expand_dims(tl.arange(0, bm), 1)) + pid_n_new * bn + tl.expand_dims(tl.arange(0, bn), 0)
    xl1_mask = n * (pid_m_new * bm + tl.expand_dims(tl.arange(0, bm), 1) + 1)
    acc = acc.to(tl.float16)
    tl.store(xl1_ptr + xl1_offset, acc, xl1_offset < xl1_mask)

    if pid_n_new==0:
        xptr_offset_1=k *(bm*pid_m_new + tl.expand_dims(tl.arange(0, bm),1)) + 2*tl.expand_dims(tl.arange(0, group_size), 0)
        xptr_offset_2=k *(bm*pid_m_new + tl.expand_dims(tl.arange(0, bm),1)) + 2*tl.expand_dims(tl.arange(0, group_size), 0)+1
        x_ptr_mask = k * tl.expand_dims(pid_m_new * bm + tl.arange(1, bm + 1), 1)
        x_ptr_mask=tl.where(x_ptr_mask<m*k , x_ptr_mask , m*k)

        
        x_save_offset=k//2 *(bm*pid_m_new + tl.expand_dims(tl.arange(0, bm),1)) + tl.expand_dims(tl.arange(0, group_size), 0)
        x_save_mask = k//2 * tl.expand_dims(pid_m_new * bm + tl.arange(1, bm + 1), 1)
        x_save_mask=tl.where(x_save_mask<m*k//2 , x_ptr_mask , m*k//2)
        
        x_scale_offset=tl.cdiv(k,2*group_size)*(bm*pid_m_new + tl.expand_dims(tl.arange(0, bm),1))
        x_scale_offset_mask=tl.cdiv(k,2*group_size)*(bm*pid_m_new + tl.expand_dims(tl.arange(1, bm+1),1))
        x_scale_offset_mask=tl.where(x_scale_offset_mask<m*tl.cdiv(k,group_size) , x_ptr_mask , m*tl.cdiv(k,group_size))

        for i in range(0, tl.cdiv(k, 2*group_size)):
            x_loaded_1=tl.load(x_ptr+xptr_offset_1 ,xptr_offset_1<x_ptr_mask ,0)
            x_loaded_2=tl.load(x_ptr+xptr_offset_2 ,xptr_offset_2<x_ptr_mask ,0)
            
            max_val_1=tl.max(tl.abs(x_loaded_1) , axis=1 , keep_dims=True)
            max_val_2=tl.max(tl.abs(x_loaded_2) , axis=1 , keep_dims=True)
            max_val=tl.where(max_val_1>max_val_2 , max_val_1 , max_val_2)
            scale = max_val/7.0 
            scaled_1 = x_loaded_1 / scale
            scaled_2 = x_loaded_2 / scale
            clamped_vals_1 = tl.clamp(scaled_1, -8.0, 7.0)
            clamped_vals_2 = tl.clamp(scaled_2, -8.0, 7.0)
            int4_vals_1 = tl.where(clamped_vals_1 >= 0,
                                 tl.floor(clamped_vals_1 + 0.5),
                                 tl.ceil(clamped_vals_1 - 0.5)).to(tl.int8)
                
            int4_vals_2 = tl.where(clamped_vals_2 >= 0,
                                 tl.floor(clamped_vals_2 + 0.5),
                                 tl.ceil(clamped_vals_2 - 0.5)).to(tl.int8)
            int8block = int4_vals_1 & 0x0F  # Lower 4 bits
            int8block2 = int4_vals_2 & 0x0F  # Upper 4 bits
            packed = (int8block2.to(tl.int8) << 4) | int8block.to(tl.int8)
            packed=packed.to(tl.int8)
            tl.store(x_quant_ptr +x_save_offset,packed  , x_save_offset<x_save_mask)
            tl.store(scale_ptr+x_scale_offset ,scale ,  x_scale_offset<x_scale_offset_mask)
    
            xptr_offset_1+=2*group_size
            xptr_offset_2+=2*group_size
            x_save_offset+=group_size
            x_scale_offset+=1

In [11]:
import torch
import torch.nn as nn
import torch
import torch.nn as nn
import gc

class SDV_FC_4b(nn.Module):
    def __init__(self, l1, l2, bias, remainder, block_size, q_bit=4):
        """
        Initializes the model.
        
        Args:
            l1, l2, bias: Standard parameters.
            remainder: The tensor to be quantized in 4-bit.
            block_size: The block size used for per-block quantization.
            q_bit: Should be 4.
        """
        super(SDV_FC_4b, self).__init__()
        self.l1 = nn.Parameter(l1.clone()).contiguous()
        self.l2 = nn.Parameter(l2.clone()).contiguous()
        self.bias = nn.Parameter(bias.clone()).contiguous()
        self.block_size = block_size
        
        if q_bit != 4:
            raise ValueError("This model is set up for 4-bit quantization only.")
        
        # Quantize the remainder using 4-bit quantization.
        # print(remainder.shape)
        quantized_remainder, scales, original_shape = quantize_vertical_pytorch(remainder, block_size)
        # print(quantized_remainder.shape , scales.shape  , original_shape)
        # Save the packed quantized tensor and scales as buffers.
        self.register_buffer('quantized_remainder', quantized_remainder.contiguous())
        self.register_buffer('scales', scales.contiguous())
        # Save the original shape as an attribute (for dequantization)
        self.original_shape = original_shape
        
        # Optionally delete the original tensors to free memory.
        del l1, l2, bias, remainder
        self.bm, self.bn, self.bk = 64, 64, 64  # Tile sizes
        self.group_sz = 4  # For swizzling
    def forward(self, x):
        # Compute the standard low-rank output.
        x=x.contiguous()
        print(x.shape)
        b,m, k = x.shape
        rank , n =self.l2.shape
        xl1 = torch.empty((b,m, rank), dtype=torch.float16, device=x.device).contiguous()
        q_x = torch.empty((b,m, k // 2), dtype=torch.int8, device=x.device).contiguous()
        s_x = torch.empty((b,m, k // self.block_size), dtype=torch.float16, device=x.device).contiguous()
        output = torch.empty((b,m, n), dtype=torch.float16, device=x.device).contiguous()
        
        # Kernel 1
        grid = lambda meta: (triton.cdiv(m, meta['bm']), triton.cdiv(n, meta['bn']))
        kernel1[grid](
            x, self.l1.contiguous(), xl1, q_x, s_x,
            m, rank, k,
            self.bm, self.bn, self.bk, self.group_sz, self.bk, self.block_size//2
        )



        
        grid = (triton.cdiv(m, self.bm), triton.cdiv(n, self.bn))
        kernel2[grid](
            xl1, q_x, s_x, self.quantized_remainder.contiguous(), self.scales.contiguous(), self.l2.contiguous(), output,
            m, n, k, rank,
            self.bm, self.bn, self.bk, self.group_sz, self.block_size
        )
        
        output+=self.bias
        torch.cuda.synchronize()
        return output


def SDV_FC_FullyConnectedLayers(fc_weight, fc_bias, block_size=64 , q_bit=4,rank=None):
    # Compute SVD
    U, S, Vh = torch.linalg.svd(fc_weight.float(), full_matrices=False)
    
    # Verify S is sorted
    assert torch.all(S[:-1] >= S[1:]), "Singular values not sorted!"
    
    # Take components corresponding to non-zero singular values
    U_r = U[:, :rank].to(torch.float16)
    S_r = S[:rank].to(torch.float16)
    V_r = Vh[:rank, :].to(torch.float16)
    
    # Create scaled matrices
    l1 = U_r @ torch.diag(S_r)
    l2 = V_r
    remainder = fc_weight-l1 @ l2
    del U, S, Vh, fc_weight
    torch.cuda.empty_cache()  # Clear GPU cache
    gc.collect()  # Force garbage collection
    return SDV_FC_4b(l1, l2, fc_bias, remainder , block_size , q_bit)


In [19]:
# Test Script
if __name__ == "__main__":
    # Create a linear layer
    in_features, out_features = 128, 64
    # torch.manual_seed(42)
    linear = nn.Linear(in_features, out_features, bias=True, dtype=torch.float16)
    linear = linear.cuda() if torch.cuda.is_available() else linear
    
    # Convert to original SDV_FC_4b
    block_size = 64
    rank = 32
    svd_model = SDV_FC_FullyConnectedLayers(linear.weight.t(), linear.bias, block_size=block_size, rank=rank)
    svd_model = svd_model.cuda() if torch.cuda.is_available() else svd_model
    
    # Input
    batch_size = 32
    X = torch.randn((1 , batch_size, in_features), dtype=torch.float16)
    X = X.cuda() if torch.cuda.is_available() else X
    
    # Reference output
    with torch.no_grad():
        output_ref = linear(X)
    
    # Original SDV-4b output
    with torch.no_grad():
        output_svd  = svd_model(X)
    
    # Compute MAE
    mae = torch.mean(torch.abs(output_ref - output_svd)).item()
    mse = torch.mean((output_ref - output_svd) ** 2).item()
    max_error = torch.max(torch.abs(output_ref - output_svd)).item()
    
    print(f"Reference output shape: {output_ref.shape}")
    print(f"Original SDV-4b output shape: {output_svd.shape}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Max Absolute Error: {max_error:.6f}")
    print("\nSample comparison:")
    print("Reference [0, 0:5]:", output_ref[0, 0:50])
    print("Original SDV-4b [0, 0:5]:", output_svd[0, 0:50])

torch.Size([1, 32, 128])
Reference output shape: torch.Size([1, 32, 64])
Original SDV-4b output shape: torch.Size([1, 32, 64])
Mean Absolute Error (MAE): 0.033051
Mean Squared Error (MSE): 0.001771
Max Absolute Error: 0.150879

Sample comparison:
Reference [0, 0:5]: tensor([[-0.3777,  0.5752, -0.3125,  ..., -0.3320, -0.0279,  0.2290],
        [-0.6294, -0.5142, -0.4915,  ...,  0.0657, -0.7607, -0.7212],
        [ 0.0692,  0.3950, -0.7295,  ..., -0.3447, -0.5454, -0.1727],
        ...,
        [ 0.0594,  1.1201,  0.2103,  ...,  0.8652, -0.7363,  0.1914],
        [-0.7119, -0.2654, -0.2028,  ...,  0.7017,  0.6855, -0.2654],
        [ 0.3623, -0.7251,  0.3799,  ..., -0.5312,  0.3713,  0.0534]],
       device='cuda:0', dtype=torch.float16)
Original SDV-4b [0, 0:5]: tensor([[-0.3850,  0.6025, -0.2412,  ..., -0.3877, -0.0434,  0.1979],
        [-0.5508, -0.4666, -0.4854,  ...,  0.1514, -0.7212, -0.6665],
        [ 0.0818,  0.4075, -0.7227,  ..., -0.2996, -0.6304, -0.0957],
        ...,
     

In [ ]:
quantize_x[0] , q_x [0]

In [ ]:
quantised_x_scale[0], s_x[0]

In [164]:
s_x.shape

torch.Size([32, 2])

In [165]:
quantised_x_scale.shape

torch.Size([32, 2])

In [33]:
output_ref[0][0]

tensor([-0.1302,  0.0868,  0.0211,  0.3508, -0.3191, -0.0359,  0.2839,  0.3750,
         0.4250, -0.1381, -0.2455, -0.2252,  0.4373, -0.9390,  0.3901,  0.4358,
        -0.4473, -0.5713,  0.2271, -0.2559,  0.0961,  0.8008,  0.7886, -0.2399,
         0.2240, -0.8550, -0.3264, -0.6255,  0.5991,  0.1028, -0.1655,  0.7212,
         0.3364,  0.7134,  0.0814,  0.5317, -0.0779, -0.2629, -0.3921,  0.6846,
         0.1550, -0.0032, -0.4170,  0.4668, -0.6440,  0.0665,  0.8735, -0.1682,
         0.0941, -1.6230,  0.2615, -0.0693, -0.7593, -0.8745,  0.7832,  0.6279,
        -0.1840,  0.1765,  0.9956, -0.0634, -0.2445,  0.5762,  0.4692,  0.1477],
       device='cuda:0', dtype=torch.float16)

In [34]:

output_svd[0]

tensor([-0.0723,  0.0812,  0.1041,  0.2749, -0.2637, -0.0202,  0.2369,  0.2637,
         0.4670, -0.1399, -0.2368, -0.2313,  0.4729, -1.0547,  0.4688,  0.3823,
        -0.4734, -0.5503,  0.1511, -0.2795,  0.0453,  0.7144,  0.6338, -0.3203,
         0.2321, -1.0332, -0.2786, -0.5078,  0.6304,  0.1066, -0.2627,  0.7983,
         0.3540,  0.6733,  0.1161,  0.4263, -0.1656, -0.3613, -0.3743,  0.6279,
         0.2372, -0.0094, -0.4221,  0.4543, -0.6274,  0.0949,  0.9268, -0.2263,
         0.1639, -1.7168,  0.2527,  0.0328, -0.7573, -0.9600,  0.7036,  0.5586,
        -0.2520,  0.0994,  1.0420, -0.0630, -0.2983,  0.6060,  0.5415,  0.2336],
       device='cuda:0', dtype=torch.float16)

In [23]:
import time
start=time.time()
for i in range(0,1000):
    print(i)
print(time.time()-start)
start=time.time()
j=0
for i in range(0,1000):
    # print(i)
    i=j+1
print(time.time()-start)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27